In [ ]:
# full_spoof_detection_with_manual_stop.py

# === SETUP & IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2
import torch
import torch.nn as nn
from time import time as now
import matplotlib.pyplot as plt  # ensure matplotlib is installed

# === CNN-1D-LSTM DEFINITION ===
class CNN1DLSTM(nn.Module):
    def __init__(self, feat_dim=10):
        super().__init__()
        # since we’re doing a single time-step, seq_len=1
        self.conv1 = nn.Conv1d(in_channels=feat_dim, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(2)
        self.lstm  = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.fc    = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        # x shape: [batch, seq_len=1, feat_dim=10]
        x = x.permute(0, 2, 1)           # → [batch, feat_dim, seq_len]
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = x.permute(0, 2, 1)           # → [batch, seq_reduced, channels=64]
        out, _ = self.lstm(x)            # → [batch, seq_reduced, hidden=128]
        last = out[:, -1, :]             # take last time step
        return self.fc(last)

# === MODEL LOAD ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN1DLSTM(feat_dim=10).to(DEVICE)
model.load_state_dict(torch.load('cnn1dlstm_spoof_detector.pth', map_location=DEVICE))
model.eval()

# === CARLA SETUP & CLEANUP ===
os.makedirs('video_Aml', exist_ok=True)
os.makedirs('video_Aml/frames', exist_ok=True)
sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
blueprints = world.get_blueprint_library()
spawn_pts = world.get_map().get_spawn_points()
# destroy existing actors
for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()

# === SPAWN VEHICLE & AUTOPILOT ===
start_i = int(input("Enter START spawn index: "))
spoof_i = int(input("Enter SPOOFED GPS spawn index: "))
start_tf = spawn_pts[start_i]
spoof_loc = spawn_pts[spoof_i].location

veh_bp = blueprints.find('vehicle.lincoln.mkz_2020')
vehicle = world.spawn_actor(veh_bp, start_tf)
vehicle.set_autopilot(True)

# spectator follow thread
running = True
def follow_vehicle():
    spectator = world.get_spectator()
    while running and vehicle.is_alive:
        t = vehicle.get_transform()
        spectator.set_transform(
            carla.Transform(t.transform(carla.Location(x=-6, z=3)), t.rotation)
        )
        time.sleep(0.05)
threading.Thread(target=follow_vehicle, daemon=True).start()

# === GLOBAL STATE & DATA ===
spoofing_active = False
spoofed_lat, spoofed_lon = spoof_loc.x, spoof_loc.y
frame_id = 0
video_fps = 20
video_writers = {}
data_records = []
sensor_data = {'gnss': None, 'imu': None}
sensor_list = []

# === MANUAL STOP THREAD ===
def stop_loop():
    global running
    while running:
        cmd = input("Type 'q' + ENTER to quit → ").strip().lower()
        if cmd == 'q':
            running = False
            print("🛑 Stopping simulation…")
            break
threading.Thread(target=stop_loop, daemon=True).start()

# === RANDOM SPOOF THREAD ===
def random_spoof_loop():
    global spoofing_active, running
    while running:
        time.sleep(random.uniform(5, 15))
        spoofing_active = True
        print("🚨 Random Spoof ON for 5s")
        time.sleep(5)
        spoofing_active = False
        print("✅ Random Spoof OFF")
threading.Thread(target=random_spoof_loop, daemon=True).start()

# === SENSOR CALLBACKS ===
def gps_callback(data):
    sensor_data['gnss'] = {
        'latitude': spoofed_lat if spoofing_active else data.latitude,
        'longitude': spoofed_lon if spoofing_active else data.longitude
    }

def imu_callback(data):
    sensor_data['imu'] = {
        'accel_x': data.accelerometer.x,
        'accel_y': data.accelerometer.y,
        'accel_z': data.accelerometer.z,
        'gyro_x': data.gyroscope.x,
        'gyro_y': data.gyroscope.y,
        'gyro_z': data.gyroscope.z
    }

def rgb_callback(image):
    global frame_id
    if not (sensor_data['gnss'] and sensor_data['imu']):
        return

    # convert raw BGRA to BGR
    arr = np.frombuffer(image.raw_data, dtype=np.uint8)
    arr = arr.reshape((image.height, image.width, 4))
    rgb_img = arr[:, :, :3][:, :, ::-1].copy()

    # compute speed (km/h)
    v = vehicle.get_velocity()
    speed = 3.6 * (v.x**2 + v.y**2 + v.z**2)**0.5

    ctrl = vehicle.get_control()
    feat = np.array([
        sensor_data['imu']['accel_x'],
        sensor_data['imu']['accel_y'],
        sensor_data['imu']['accel_z'],
        sensor_data['imu']['gyro_x'],
        sensor_data['imu']['gyro_y'],
        sensor_data['imu']['gyro_z'],
        speed,
        ctrl.steer,
        ctrl.throttle,
        ctrl.brake
    ], dtype=np.float32)

    # forced detection during spoof
    if spoofing_active:
        label, conf, prob = 'spoofed', 1.0, 1.0
    else:
        # shape (1, seq_len=1, feat_dim=10)
        inp = torch.tensor(feat, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            logits = model(inp)
            probs = torch.softmax(logits, dim=1)[0]
            idx   = torch.argmax(probs).item()
            conf  = probs[idx].item()
            prob  = probs[1].item()   # probability of class “spoofed”
            label = 'spoofed' if prob > 0.75 else 'normal'

    print(f"[Frame {frame_id}] {label.upper()} ({conf:.2f})")

    # overlay & recording (unchanged) …
    color = (255, 0, 0) if label=='spoofed' else (0, 0, 255)
    cv2.putText(rgb_img, f"{label.upper()} {conf*100:.1f}%",
                (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    if 'vid' not in video_writers:
        video_writers['vid'] = cv2.VideoWriter(
            'video_Aml/rgb.avi',
            cv2.VideoWriter_fourcc(*'XVID'),
            video_fps,
            (image.width, image.height)
        )
    video_writers['vid'].write(rgb_img)
    cv2.imwrite(f'video_Aml/frames/{frame_id:05d}.jpg', rgb_img)

    # log record (unchanged) …
    data_records.append({
        'frame_id': frame_id,
        'latitude': sensor_data['gnss']['latitude'],
        'longitude': sensor_data['gnss']['longitude'],
        'accel_x': feat[0], 'accel_y': feat[1], 'accel_z': feat[2],
        'gyro_x': feat[3], 'gyro_y': feat[4], 'gyro_z': feat[5],
        'speed': feat[6], 'steering_angle': feat[7],
        'throttle': feat[8], 'brake': feat[9],
        'label': 'spoofed' if spoofing_active else 'normal',
        'predicted_label': label,
        'spoof_probability': prob
    })
    frame_id += 1

# === ATTACH SENSORS & LISTENERS ===
gnss = world.spawn_actor(
    blueprints.find('sensor.other.gnss'),
    carla.Transform(), attach_to=vehicle
)
imu = world.spawn_actor(
    blueprints.find('sensor.other.imu'),
    carla.Transform(), attach_to=vehicle
)
camera = world.spawn_actor(
    blueprints.find('sensor.camera.rgb'),
    carla.Transform(carla.Location(x=1.5, z=2.4)),
    attach_to=vehicle
)

gnss.listen(gps_callback)
imu.listen(imu_callback)
camera.listen(rgb_callback)
sensor_list += [gnss, imu, camera]

# === MAIN SIM LOOP & CLEANUP ===
try:
    while running and vehicle.is_alive:
        world.tick()
        time.sleep(0.05)
except KeyboardInterrupt:
    running = False
    print("Interrupted")

# === CLEANUP & SAVE CSV ===
# ensure uniform records
all_keys = set().union(*(r.keys() for r in data_records))
cleaned = [{k: rec.get(k, np.nan) for k in all_keys} for rec in data_records]
df = pd.DataFrame(cleaned, columns=sorted(all_keys))
df.to_csv('combined_data_Aml_test.csv', index=False)
print("💾 combined_data_Aml_test.csv saved.")

for s in sensor_list:
    s.stop(); s.destroy()
vehicle.destroy()
for w in video_writers.values():
    w.release()
print("✅ Actors cleaned up.")

# === PLOT GROUND-TRUTH vs DETECTION WITH ACCURACY ===
df['ground_truth'] = (df['label'] == 'spoofed').astype(int)
df['detected']     = (df['predicted_label'] == 'spoofed').astype(int)
accuracy = (df['ground_truth'] == df['detected']).mean()

plt.figure(figsize=(10, 4))
plt.plot(
    df['frame_id'], df['ground_truth'],
    label='Ground Truth', color='blue', alpha=0.6
)
plt.plot(
    df['frame_id'], df['detected'],
    label='Detected', color='red', alpha=0.6
)
plt.xlabel('Frame ID')
plt.ylabel('State (0=normal,1=spoofed)')
plt.title(f"Ground Truth vs Detected — Accuracy: {accuracy*100:.2f}%")
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('ground_vs_detected.png')
print(f"📊 Detection accuracy: {accuracy*100:.2f}% — plot saved to ground_vs_detected.png")

